<a href="https://colab.research.google.com/github/jywleah1101/dacon_contest/blob/main/%EB%A9%94%EB%AA%A8%ED%8C%8C%EC%9D%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IMPORT DATA

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import re
import urllib.request
from tqdm import tqdm
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from google.colab import drive

In [2]:
!pip install scikit-learn
!pip install tqdm
!pip install tensorflow

In [3]:
import os
os.chdir('/content/drive/MyDrive/NH_공모전_데이터')

In [4]:
# RGS_DT : 발행일자
# TCK_IEM_CD : 종목 티커 코드
# TIL_IFO : 제목정보
# CTGY_CFC_IFO : 카테고리분류정보
# MDI_IFO : 미디어정보
# NEWS_SMY_IFO : 뉴스요약정보
# RLD_OSE_IEM_TCK_CD : 관련해외종목티커코드
# URL_IFO : URL정보

In [5]:
raw_data_01 = pd.read_csv('NASDAQ_RSS_IFO/NASDAQ_RSS_IFO_202301.csv',encoding='cp949')
raw_data_01

,rgs_dt,tck_iem_cd,til_ifo,ctgy_cfc_ifo,mdi_ifo,news_smy_ifo,rld_ose_iem_tck_cd,url_ifo
0,20230102,NGS,Hoak & Co. Urges NGS Board to Halt CEO Search ...,Investing,Fintel,"Fintel reports that Hoak Public Equities, LP, ...",NGS,https://www.nasdaq.com/articles/hoak-co.-urges...
1,20230102,PFX,Here's Why PhenixFIN (PFX) is Poised for a Tur...,"Stocks,Investing",Zacks,PhenixFIN (PFX) has been beaten down lately wi...,PFX,https://www.nasdaq.com/articles/heres-why-phen...
2,20230102,TYG,My Advice? Sell These 2 Dividend Funds in 2023,"Markets,Stocks",BNK Invest,There's a disconnect setting up in the energy ...,"TYG,TYG,FEN",https://www.nasdaq.com/articles/my-advice-sell...
3,20230102,PFX,Here's Why PhenixFIN (PFX) is Poised for a Tur...,"Stocks,Investing",Zacks,PhenixFIN (PFX) has been beaten down lately wi...,PFX,https://www.nasdaq.com/articles/heres-why-phen...
4,20230102,TYG,My Advice? Sell These 2 Dividend Funds in 2023,"Markets,Stocks",BNK Invest,There's a disconnect setting up in the energy ...,"TYG,TYG,FEN",https://www.nasdaq.com/articles/my-advice-sell...
...,...,...,...,...,...,...,...,...
55234,20230131,WF,WF vs. HDB: Which Stock Is the Better Value Op...,"Stocks,Investing",Zacks,Investors interested in stocks from the Banks ...,"WF,HDB",https://www.nasdaq.com/articles/wf-vs.-hdb:-wh...
55235,20230131,GNS,"Is the 1,600% Rise in Genius Group Stock Justi...",Markets,MarketBeat,Global Edtech and education company Genius Gro...,"GNS,GNS,STRA,CHGG,LRN",https://www.nasdaq.com/articles/is-the-1600-ri...
55236,20230131,PNT,MORNING BID-'Soft landing' or 'no landing'?,"BioTech,US Markets,Banking",Reuters,There's an uncomfortable feeling in markets th...,"JNPR,AMZN,BXP,MDLZ,EW,EA,MCD,SPOT,PNT,GOOGL,AO...",https://www.nasdaq.com/articles/morning-bid-so...
55237,20230131,PMVP,BlackRock Increases Position in PMV Pharmaceut...,Stocks,Fintel,Fintel reports that BlackRock has filed a 13G/...,PMVP,https://www.nasdaq.com/articles/blackrock-incr...


In [6]:
raw_data_02 = pd.read_csv('NASDAQ_RSS_IFO/NASDAQ_RSS_IFO_202302.csv',encoding='cp949')
raw_data_02['news_smy_ifo']

0         A few weeks back, bond giant PIMCO trimmed a b...
1         (RTTNews) - Benchmark Electronics Inc (BHE) re...
2         Fintel reports that BlackRock has filed a 13G/...
3         Here at Zacks, our focus is on the proven Zack...
4         Fintel reports that Federated Hermes has filed...
                                ...                        
102455    Technology stocks were narrowly lower Tuesday ...
102456    Financial stocks were hanging on for moderate ...
102457    The following are today's upgrades for Validea...
102458    The following are today's upgrades for Validea...
102459    (RTTNews) - The California-based biopharmaceut...
Name: news_smy_ifo, Length: 102460, dtype: object

In [7]:
raw_data_01['news_smy_ifo'][0]

'Fintel reports that Hoak Public Equities, LP, has filed a 13D/A form with the SEC disclosing ownership of 1,150,000 shares of Natural Gas Services Group, Inc. (NGS). This represents 9.2563% of the company.'

## **데이터 전처리**

In [8]:
!pip3 install nltk
import nltk

In [9]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [14]:
def data_text_cleaning(x):

    #1. 영어가 아닌 문자를 공백으로 대체
    #영어가 아닌 숫자/특수문자의 경우 감정분석 피처로는 별 의미가 없으므로 공란으로 변경
    # 파이썬의 정규 표현식 모듈인 re를 이용해 영어 문자열이 아닌 문자는 모두 공백으로 변환
    x = x.apply(lambda a : re.sub("[^a-zA-Z]", " ", a))

    #2. 대문자를 소문자로 변환
    #모든 단어를 소문자로 변환(영어의 경우 문장의 시작/고유명사는 대문자로 시작하여 분석할 때 다른 단어로 취급)
    x = x.apply(lambda a : a.lower())

    #불용어 제거 및 어간 추출

    from nltk.corpus import stopwords

    def process_text(text):
          #3. 불용어 제거
          stop_words = set(stopwords.words('english'))
          cleaned_text = ' '.join([word for word in text.split() if word not in stop_words])

          #4. 어간 추출
          #see, saw, seen / run, running, run 등 과거형/미래형을 하나의 단어로 통일하는 작업
          stemmer = nltk.stem.SnowballStemmer('english')
          cleaned_text = ' '.join([stemmer.stem(word) for word in cleaned_text.split()])

          return cleaned_text

    x = x.apply(process_text)

    # 원본 데이터를 변경
    raw_data_01['news_smy_ifo'] = x

    return x

In [15]:
data_text_cleaning(raw_data_01['news_smy_ifo'])

0        fintel report hoak public equiti lp file form ...
1        phenixfin pfx beaten late much sell pressur st...
2        disconnect set energi market flash yellow ligh...
3        phenixfin pfx beaten late much sell pressur st...
4        disconnect set energi market flash yellow ligh...
                               ...                        
55234    investor interest stock bank foreign sector pr...
55235    global edtech educ compani genius group ltd ny...
55236    uncomfort feel market week good news may bad n...
55237    fintel report blackrock file g form sec disclo...
55238    berenson acquisit corp announc today decis can...
Name: news_smy_ifo, Length: 55239, dtype: object

In [17]:
raw_data_01

,rgs_dt,tck_iem_cd,til_ifo,ctgy_cfc_ifo,mdi_ifo,news_smy_ifo,rld_ose_iem_tck_cd,url_ifo
0,20230102,NGS,Hoak & Co. Urges NGS Board to Halt CEO Search ...,Investing,Fintel,fintel report hoak public equiti lp file form ...,NGS,https://www.nasdaq.com/articles/hoak-co.-urges...
1,20230102,PFX,Here's Why PhenixFIN (PFX) is Poised for a Tur...,"Stocks,Investing",Zacks,phenixfin pfx beaten late much sell pressur st...,PFX,https://www.nasdaq.com/articles/heres-why-phen...
2,20230102,TYG,My Advice? Sell These 2 Dividend Funds in 2023,"Markets,Stocks",BNK Invest,disconnect set energi market flash yellow ligh...,"TYG,TYG,FEN",https://www.nasdaq.com/articles/my-advice-sell...
3,20230102,PFX,Here's Why PhenixFIN (PFX) is Poised for a Tur...,"Stocks,Investing",Zacks,phenixfin pfx beaten late much sell pressur st...,PFX,https://www.nasdaq.com/articles/heres-why-phen...
4,20230102,TYG,My Advice? Sell These 2 Dividend Funds in 2023,"Markets,Stocks",BNK Invest,disconnect set energi market flash yellow ligh...,"TYG,TYG,FEN",https://www.nasdaq.com/articles/my-advice-sell...
...,...,...,...,...,...,...,...,...
55234,20230131,WF,WF vs. HDB: Which Stock Is the Better Value Op...,"Stocks,Investing",Zacks,investor interest stock bank foreign sector pr...,"WF,HDB",https://www.nasdaq.com/articles/wf-vs.-hdb:-wh...
55235,20230131,GNS,"Is the 1,600% Rise in Genius Group Stock Justi...",Markets,MarketBeat,global edtech educ compani genius group ltd ny...,"GNS,GNS,STRA,CHGG,LRN",https://www.nasdaq.com/articles/is-the-1600-ri...
55236,20230131,PNT,MORNING BID-'Soft landing' or 'no landing'?,"BioTech,US Markets,Banking",Reuters,uncomfort feel market week good news may bad n...,"JNPR,AMZN,BXP,MDLZ,EW,EA,MCD,SPOT,PNT,GOOGL,AO...",https://www.nasdaq.com/articles/morning-bid-so...
55237,20230131,PMVP,BlackRock Increases Position in PMV Pharmaceut...,Stocks,Fintel,fintel report blackrock file g form sec disclo...,PMVP,https://www.nasdaq.com/articles/blackrock-incr...


In [18]:
#단어들만 남은 것 확인 가능
raw_data_01["news_smy_ifo"][0]

'fintel report hoak public equiti lp file form sec disclos ownership share natur gas servic group inc ngs repres compani'


```
# **EDA처리 전** ; raw_data_01["news_smy_ifo"][0]
Fintel reports that Hoak Public Equities, LP, has filed a 13D/A form with the SEC disclosing ownership of 1,150,000 shares of Natural Gas Services Group, Inc. (NGS). This represents 9.2563% of the company.
```
